In [1]:
import pandas as pd
def calculate_indicator_signals(df: pd.DataFrame, indicator_name, variables, detect_divergence=False):
    signals = {"indicator": indicator_name, "divergence_detected": False, "side": None}

    if indicator_name == "MACD":
        fast_length = variables.get("fast_length", 12)
        slow_length = variables.get("slow_length", 26)
        signal_length = variables.get("signal_length", 9)

        df["EMA_fast"] = df["Close"].ewm(span=fast_length, adjust=False).mean()
        df["EMA_slow"] = df["Close"].ewm(span=slow_length, adjust=False).mean()
        df["MACD_line"] = df["EMA_fast"] - df["EMA_slow"]
        df["MACD_signal"] = df["MACD_line"].ewm(span=signal_length, adjust=False).mean()
        df["MACD_hist"] = df["MACD_line"] - df["MACD_signal"]
        signals["last_value"] = df["MACD_hist"].iloc[-1]

        if detect_divergence:
            if df["MACD_hist"].iloc[-1] > 0 and df["MACD_hist"].iloc[-2] <= 0:
                signals["divergence_detected"] = True
                signals["side"] = "BUY"
            elif df["MACD_hist"].iloc[-1] < 0 and df["MACD_hist"].iloc[-2] >= 0:
                signals["divergence_detected"] = True
                signals["side"] = "SELL"

    elif indicator_name == "RSI":
        length = variables.get("length", 14)
        delta = df["Close"].diff()
        gain = (delta.where(delta > 0, 0)).rolling(window=length).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(window=length).mean()
        rs = gain / loss
        df["RSI"] = 100 - (100 / (1 + rs))
        signals["last_value"] = df["RSI"].iloc[-1]

        if detect_divergence:
            if df["RSI"].iloc[-1] < 30:
                signals["divergence_detected"] = True
                signals["side"] = "BUY"
            elif df["RSI"].iloc[-1] > 70:
                signals["divergence_detected"] = True
                signals["side"] = "SELL"

    return df, signals

### Tweaking with Binance

In [1]:
import os
from binance.client import Client
import pandas as pd

# Test 
api_key = os.environ['BINANCE_API_KEY']
api_secret = os.environ['BINANCE_SECRET_KEY']
client = Client(api_key, api_secret, testnet=True)
tickers = client.get_all_tickers()

# EZ!
pd.DataFrame(tickers)

ModuleNotFoundError: No module named 'binance.client'

In [ ]:
# fetch 30 minute klines for the last month
klines = client.get_historical_klines(symbol="BNBBTC", interval=Client.KLINE_INTERVAL_1DAY, start_str="1 Dec, 2023", end_str="1 Jan, 2024")
pd.DataFrame(klines)

In [ ]:
lient = Client(api_key, api_secret)

# Fetch historical kline data
klines = client.get_historical_klines("BNBBTC", Client.KLINE_INTERVAL_1MINUTE, "1 day ago UTC")

# Convert to DataFrame and select OHLC columns
df = pd.DataFrame(klines, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_asset_volume', 'number_of_trades', 'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore'])
ohlc_df = df[['timestamp', 'open', 'high', 'low', 'close', 'volume']]

# Convert timestamp to datetime
ohlc_df['timestamp'] = pd.to_datetime(ohlc_df['timestamp'], unit='ms')

# Display the DataFrame
ohlc_df

In [5]:
import pandas as pd
import os
from binance.spot import Spot

 
api_key = os.environ['BINANCE_API_KEY']
api_secret = os.environ['BINANCE_SECRET_KEY']

# api key/secret are required for user data endpoints
client = Spot(base_url="https://api1.binance.com", api_key=api_key, api_secret=api_secret)
# Get server timestamp
print(client.time())
# Get last 10 klines of BNBUSDT at 1h interval
print(client.klines("BNBUSDT", "1h", limit=10))



#  Get candlestick data for BNBUSDT at 1h interval
klines = client.klines("BNBUSDT", "1h", limit=10)


# Convert to DataFrame and select OHLC columns
df = pd.DataFrame(klines, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_asset_volume', 'number_of_trades', 'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore'])
ohlc_df = df[['timestamp', 'open', 'high', 'low', 'close', 'volume']]

# Convert timestamp to datetime
ohlc_df['timestamp'] = pd.to_datetime(ohlc_df['timestamp'], unit='ms')
ohlc_df

{'serverTime': 1736839847352}
[[1736805600000, '683.00000000', '688.00000000', '682.46000000', '686.48000000', '10496.63800000', 1736809199999, '7193525.67352000', 22454, '5893.40900000', '4038650.26379000', '0'], [1736809200000, '686.47000000', '688.71000000', '685.43000000', '688.64000000', '4286.39000000', 1736812799999, '2945047.96036000', 11314, '2621.77100000', '1801063.73526000', '0'], [1736812800000, '688.64000000', '689.73000000', '686.30000000', '687.47000000', '5000.87100000', 1736816399999, '3441059.45370000', 16588, '2238.04600000', '1540519.35521000', '0'], [1736816400000, '687.47000000', '690.99000000', '687.46000000', '688.91000000', '6796.93200000', 1736819999999, '4686777.77041000', 21487, '3709.36100000', '2557823.70203000', '0'], [1736820000000, '688.91000000', '692.00000000', '688.31000000', '690.20000000', '6508.64900000', 1736823599999, '4494486.04881000', 18477, '4175.85800000', '2884061.54804000', '0'], [1736823600000, '690.20000000', '692.50000000', '690.04000

C:\Users\crusa\AppData\Local\Temp\ipykernel_23220\2741385975.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ohlc_df['timestamp'] = pd.to_datetime(ohlc_df['timestamp'], unit='ms')


,timestamp,open,high,low,close,volume
0,2025-01-13 22:00:00,683.00000000,688.00000000,682.46000000,686.48000000,10496.63800000
1,2025-01-13 23:00:00,686.47000000,688.71000000,685.43000000,688.64000000,4286.39000000
2,2025-01-14 00:00:00,688.64000000,689.73000000,686.30000000,687.47000000,5000.87100000
3,2025-01-14 01:00:00,687.47000000,690.99000000,687.46000000,688.91000000,6796.93200000
4,2025-01-14 02:00:00,688.91000000,692.00000000,688.31000000,690.20000000,6508.64900000
5,2025-01-14 03:00:00,690.20000000,692.50000000,690.04000000,692.02000000,2765.08000000
6,2025-01-14 04:00:00,692.02000000,693.00000000,688.99000000,689.83000000,9448.12500000
7,2025-01-14 05:00:00,689.82000000,691.20000000,687.62000000,687.77000000,5397.97100000
8,2025-01-14 06:00:00,687.77000000,690.80000000,687.33000000,690.80000000,4812.33600000
9,2025-01-14 07:00:00,690.80000000,691.20000000,689.40000000,690.14000000,2603.33900000


## Pandas and Stock Market Data

In [3]:
import pandas as pd

df = pd.read_csv("synthetic_stock_data.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 252 entries, 0 to 251
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Date    252 non-null    object 
 1   Close   252 non-null    float64
 2   Volume  252 non-null    int64  
 3   Open    252 non-null    float64
 4   High    252 non-null    float64
 5   Low     252 non-null    float64
dtypes: float64(4), int64(1), object(1)
memory usage: 11.9+ KB


In [7]:
!pip install TA-lib==0.6.0

  Using cached ta_lib-0.6.0.tar.gz (371 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
Failed to build TA-lib


  error: subprocess-exited-with-error
  
  × Building wheel for TA-lib (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [348 lines of output]
      <string>:75: UserWarning: Cannot find ta-lib library, installation may fail.
      C:\Users\crusa\AppData\Local\Temp\pip-build-env-mlavpvgv\overlay\Lib\site-packages\setuptools\config\_apply_pyprojecttoml.py:81: SetuptoolsWarning: `install_requires` overwritten in `pyproject.toml` (dependencies)
        corresp(dist, value, root_dir)
      running bdist_wheel
      running build
      running build_py
      creating build\lib.win-amd64-cpython-312\talib
      copying talib\abstract.py -> build\lib.win-amd64-cpython-312\talib
      copying talib\deprecated.py -> build\lib.win-amd64-cpython-312\talib
      copying talib\stream.py -> build\lib.win-amd64-cpython-312\talib
      copying talib\__init__.py -> build\lib.win-amd64-cpython-312\talib
      running egg_info
      writing ta_lib.egg-info\PKG-INFO
      writing dependenc

In [4]:
import talib as ta
import numpy as np

df['sma9'] = ta.SMA((df['Close']), 5)
df['sma21'] = ta.SMA((df['Close']), 21)

ModuleNotFoundError: No module named 'talib'